In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split  # 用于切分数据
from sklearn.preprocessing import StandardScaler     # 用于数据标准化
import torch.optim as optim

print("torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

torch version: 2.5.1
CUDA available: True


In [8]:
# 1. 读数据
train = pd.read_csv("data/train.csv")


# 2. 只挑几个数值型特征（先简单）
features = ["GrLivArea", "OverallQual", "GarageCars"]  # 居住面积、房屋质量、车库容量
X = train[features].values
y = train["SalePrice"].values  


# 3. 对目标取对数
y_log = np.log1p(y)


# 4. 切分训练/验证集
X_tr, X_val, y_tr, y_val = train_test_split(X, y_log, test_size=0.2, random_state=42)


# 5. 标准化特征
# 只用训练集拟合 StandardScaler，然后分别变换训练集与验证集
scaler = StandardScaler()  # 创建一个标准化器对象。默认 with_mean=True, with_std=True，按列（特征维）做标准化
X_tr = scaler.fit_transform(X_tr)
# fit：只在训练集 X_tr 上计算每一列的 mean_ 与 scale_（等于标准差，sqrt(var_)）
# 并存到对象里（属性：scaler.mean_, scaler.scale_）。
# transform：用刚学到的 mean_ 和 scale_ 立刻把 X_tr 变换为标准化后的数据。
# 合起来的 fit_transform：一步做完，返回的是已标准化的训练集（还是 NumPy 数组）
X_val = scaler.transform(X_val)
# 用训练集的均值与标准差把验证集做同样的线性变换，模拟真实部署场景。


# 6. 转张量
X_tr = torch.tensor(X_tr, dtype=torch.float32)
y_tr = torch.tensor(y_tr, dtype=torch.float32).reshape(-1, 1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).reshape(-1, 1)

print("y_tr 前 5 个:", y_tr[:5].view(-1))

y_tr 前 5 个: tensor([11.8845, 12.0895, 11.3504, 12.0725, 11.7520])


In [9]:
# 定义模型（还是 MLP：输入->隐藏层->ReLU->输出）
input_dim = X_tr.shape[1]
model = nn.Sequential(
    nn.Linear(input_dim, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 训练循环
num_epochs = 20000
for epoch in range(num_epochs):
    model.train()
    y_pred = model(X_tr)                # 预测 log 房价
    loss = criterion(y_pred, y_tr)      # 和 log(y) 对比

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]  Train MSE(log): {loss.item():.4f}")

Epoch [20/20000]  Train MSE(log): 135.1636
Epoch [40/20000]  Train MSE(log): 129.6004
Epoch [60/20000]  Train MSE(log): 123.7980
Epoch [80/20000]  Train MSE(log): 117.5928
Epoch [100/20000]  Train MSE(log): 110.9079
Epoch [120/20000]  Train MSE(log): 103.7423
Epoch [140/20000]  Train MSE(log): 96.1513
Epoch [160/20000]  Train MSE(log): 88.2502
Epoch [180/20000]  Train MSE(log): 80.1952
Epoch [200/20000]  Train MSE(log): 72.1598
Epoch [220/20000]  Train MSE(log): 64.3005
Epoch [240/20000]  Train MSE(log): 56.7860
Epoch [260/20000]  Train MSE(log): 49.7805
Epoch [280/20000]  Train MSE(log): 43.4361
Epoch [300/20000]  Train MSE(log): 37.8501
Epoch [320/20000]  Train MSE(log): 33.0660
Epoch [340/20000]  Train MSE(log): 29.0722
Epoch [360/20000]  Train MSE(log): 25.8023
Epoch [380/20000]  Train MSE(log): 23.1555
Epoch [400/20000]  Train MSE(log): 21.0158
Epoch [420/20000]  Train MSE(log): 19.2654
Epoch [440/20000]  Train MSE(log): 17.7973
Epoch [460/20000]  Train MSE(log): 16.5372
Epoch [48

In [ ]:
# 验证集评估
model.eval()
with torch.no_grad():  # 上下文不跟踪梯度，前向计算不建立计算图，不存梯度
    # log 空间 RMSE（比赛用）
    val_pred_log = model(X_val)
    val_mse_log = criterion(val_pred_log, y_val).item()
    # .item() 把 0 维张量（标量）取成 Python 浮点数，方便打印/记录。
    val_rmse_log = val_mse_log ** 0.5

print(f"Validation RMSE (log space): {val_rmse_log:.4f}")

Validation RMSE (log space): 0.1930
